In [16]:
import os
from dotenv import load_dotenv
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationStringBufferMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
import omegaconf

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read data from YAML file

The instructions and examples to build up a prompt based on few-shot
examples are kept in a configuration YAML file.

This code read data from a YAML file using the `OmegaConf` library, and
convert an `OmegaConf` object to a standard Python list.

This line loads the YAML file located at `"../config/main.yaml"` into a `config` object. The `config` object is an `OmegaConf` container that allows for easy access to the data in the YAML file.

In [18]:
# Load the config file
config = omegaconf.OmegaConf.load("../config/main.yaml")

These lines access the data associated with the keys `"instructions"` and `"examples"` in the YAML file. The `instructions_text` variable will hold the text data, and `examples` will hold an `OmegaConf.ListConfig` object containing the list of dictionaries.

In [19]:
# Access the examples in the yaml file
examples = config["examples"]

In [20]:
# Assuming examples is your ListConfig object
examples = omegaconf.OmegaConf.to_container(examples, resolve=True)

### Fixed few-shot prompting

this approach follows LangChain [few-shot examples for chat models docs](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples_chat):

In [21]:
load_dotenv()

openai.api_key = os.environ["OPENAI_API_KEY"]

In [22]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Generate a piece of music in ABC notation inspired by Scott Joplin's
ragtime style. The piece should have syncopated rhythms with emphasis on
off-beats, a structured form similar to A-A-B-B-A-C-C-D-D with each
section being 16 bars, bright and memorable melodies, tonal harmonic
progressions with chromatic passing tones and secondary dominants, and a
steady 'oom-pah' bass pattern in the left hand. Include expressive
markings, dynamic changes, and tempo indications to capture the spirited
and expressive quality of ragtime.

AI: X:357
T:The Entertainer
C:Scott Joplin, 1902, arranged Colin Hume
L:1/8
M:2/2
R:Rag
S:Colin Hume's website,  colinhume.com  - chords can also be printed below the stave.
Q:1/2=76
%%MIDI chordname dim 0 3 6 9
N:For the dance "The Heathfield Rag" by Colin Hume
K:C
P:Intro
[| d'e'c'a-abg2 | decA-ABG2 | DECA,-A,B,A,_A, | G,2 z2 [G,B,DG]2 ||
P:Tune 1
|: D^D | "C"Ec-cE "C7"c2Ec- | "F"c4- "C/E"ccd^d | "C"ecde- "G7"eBd2 | "C"c6 D^D |
"C"Ec-cE "C7"c2Ec- | "F"c4- "C/

In [23]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of music creation."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

- `temperature` number or null Optional Defaults to 1
What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
We generally recommend altering this or `top_p` but not both.
- `top_p` number or null Optional Defaults to 1
An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
We generally recommend altering this or `temperature` but not both.
- `frequency_penalty` number or null Optional Defaults to 0
Number between -2.0 and 2.0. Positive values penalize new tokens based
on their existing frequency in the text so far, decreasing the model's
likelihood to repeat the same line verbatim.
- `presence_penalty` number or null Optional Defaults to 0
Number between -2.0 and 2.0. Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

See more information about [frequency and presence
penalties](https://platform.openai.com/docs/guides/gpt/parameter-details).

The following article clarifies the difference between Frequency and
Presence: [Frequency vs Presence penalty, what’s the difference? —
OpenAI
API](https://medium.com/@KTAsim/frequency-vs-presence-penalty-whats-the-difference-openai-api-51b0c4a7229e)

Summary:

- Frequency Penalty helps us avoid using the same words too often. It’s like telling the computer, “Hey, don’t repeat words too much.”
- Presence Penalty, on the other hand, encourages using different words. It’s like saying, “Hey, use a variety of words, not just the same ones.”
- So, Frequency Penalty stops repetition, while Presence Penalty encourages variety. They work differently but help make the text more interesting.

Frequency and Presence penalties are not sides of the same coin:

These two expressions share a common underlying aim of reducing repetition and promoting variety in word usage. However, they emphasize different aspects and are not strictly logically equivalent.

1. **“Don’t repeat words too much”**:
    - This expression directly addresses the issue of repetition. It instructs to avoid using the same words over and over, which is a straightforward directive against repetition.

2. **“Use a variety of words, not just the same ones”**:
    - This expression goes a step further by not only discouraging repetition (implied by the phrase "not just the same ones") but also actively encouraging diversity in word choice (implied by the phrase "use a variety of words"). 

The difference can be highlighted in a scenario where one adheres to the instruction of not repeating words too much but still uses a very limited vocabulary. They would be following the first expression but not the second. On the other hand, someone who uses a variety of words is likely adhering to both expressions.

In a broader sense, the second expression encompasses the directive of the first but adds an additional directive to use a wider range of vocabulary. Hence, while they align in discouraging repetition, they diverge on the aspect of promoting diversity, making them not entirely equivalent.

In [24]:
llm_model = "gpt-4"
temperature = 1
other_parameters = {"frequency_penalty": 0, "presence_penalty": 0}

In [25]:
llm = ChatOpenAI(temperature=temperature, model=llm_model, model_kwargs=other_parameters)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [26]:
chain = final_prompt | llm

style = "Scott Joplin's ragtime"

output = chain.invoke(
    {
        "input": f"Generate a piece of music in ABC notation inspired by {style} style"
    }
)

In [27]:
print(output)

content='Here is a basic piece of music in \'ABC\' Notation inspired by the ragtime stylings of Scott Joplin:\n\n```\nX:1\nT:Ragtime Reflections\nM:4/4\nL:1/8\nR:Ragtime\nK:C\n"C"c2c2 "E7"e2d2| "A7"c4       -c2d2| "D7"f2f2 "G7"g2f2| "C"e4       -e2c2|\n"C"c2c2 "E7"e2d2| "A7"a2g2     -g2f2| "D7"d2d2 "G7"c2B2| "C"c6        c2   |\n"C7"c2e2 g2e2  | "F"f4        -f2d2| "C"c2c2 "G7"BA^G^F| "G7"E4       -E2D2|\n"C"C2E2 G2E2  | "F"F4        -F2D2| "C"e2c2 "G7"d2B2 | "C"c6        c2   |\n```\n\nHere, each note represents the melody, while the letters in quotes represent the chords.' additional_kwargs={} example=False


### Dynamic few-shot prompting

this approach follows LangChain [few-shot examples for chat models docs](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples_chat):

Since we are using a vectorstore to select examples based on semantic similarity, we will want to first populate the store.

In [28]:
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

#### Create the  `example_selector`[​](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples_chat#create-the-example_selector "Direct link to create-the-example_selector")

With a vectorstore created, you can create the  `example_selector`. Here we will isntruct it to only fetch the top 2 examples.

In [29]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "Bach"})

[{'input': "Compose a simple minuet in ABC notation, embodying the style and\ngrace characteristic of J.S. Bach's dance compositions.  Each\npiece should have a time signature of 3/4, reflecting the elegant\ntriple meter typical of a minuet. The harmonic language should\nremain tonal, utilizing common practice progressions and\nmodulations.  \n\nStructure: Each minuet should have a binary form (AABB), where\neach section (A and B) consists of 8 bars. A modulation to the\ndominant or relative major/minor in the B section would be\nappreciated, returning to the tonic by the end of the piece.\n\nMelody and Rhythm: Melodies should be balanced and singable, with\na clear, simple rhythmic structure. Ornamentations such as trills\nor mordents, which are typical of the Baroque period, are\nencouraged. \n\nBass Line: A continuous bass line should underpin the harmonic\nprogressions, with a walking bass or arpeggiated figures to\nmaintain a gentle, flowing rhythm.\n\nDynamics and Articulation: I

#### Create prompt template[​](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples_chat#create-prompt-template "Direct link to Create prompt template")

Assemble the prompt template, using the  `example_selector`  created above.

In [30]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

Below is an example of how this would be assembled.

In [31]:
print(few_shot_prompt.format(input="Generate a piece of music in ABC notation inspired by Bach style"))

Human: Compose a simple minuet in ABC notation, embodying the style and
grace characteristic of J.S. Bach's dance compositions.  Each
piece should have a time signature of 3/4, reflecting the elegant
triple meter typical of a minuet. The harmonic language should
remain tonal, utilizing common practice progressions and
modulations.  

Structure: Each minuet should have a binary form (AABB), where
each section (A and B) consists of 8 bars. A modulation to the
dominant or relative major/minor in the B section would be
appreciated, returning to the tonic by the end of the piece.

Melody and Rhythm: Melodies should be balanced and singable, with
a clear, simple rhythmic structure. Ornamentations such as trills
or mordents, which are typical of the Baroque period, are
encouraged. 

Bass Line: A continuous bass line should underpin the harmonic
progressions, with a walking bass or arpeggiated figures to
maintain a gentle, flowing rhythm.

Dynamics and Articulation: Include dynamic markings an

Assemble the final prompt template:

In [32]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of music creation."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [33]:
print(few_shot_prompt.format(input="Generate a piece of music in ABC notation inspired by Bach style"))

Human: Compose a simple minuet in ABC notation, embodying the style and
grace characteristic of J.S. Bach's dance compositions.  Each
piece should have a time signature of 3/4, reflecting the elegant
triple meter typical of a minuet. The harmonic language should
remain tonal, utilizing common practice progressions and
modulations.  

Structure: Each minuet should have a binary form (AABB), where
each section (A and B) consists of 8 bars. A modulation to the
dominant or relative major/minor in the B section would be
appreciated, returning to the tonic by the end of the piece.

Melody and Rhythm: Melodies should be balanced and singable, with
a clear, simple rhythmic structure. Ornamentations such as trills
or mordents, which are typical of the Baroque period, are
encouraged. 

Bass Line: A continuous bass line should underpin the harmonic
progressions, with a walking bass or arpeggiated figures to
maintain a gentle, flowing rhythm.

Dynamics and Articulation: Include dynamic markings an


#### Use with an LLM[​](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples_chat#use-with-an-llm "Direct link to Use with an LLM")

Now, you can connect your model to the few-shot prompt.

In [34]:
chain = final_prompt | llm

style = "Bach"

output = chain.invoke(
    {
        "input": f"Generate a piece of music in ABC notation inspired by {style} style"
    }
)

In [35]:
print(output)

content='Sure, here is one more minuet:\n\nX: 7\nT: Bach Minuet 4\nC: J. S. Bach (1685-1750)\nM: 3/4\nL: 1/8\nK: C\n|: "C" G,ECC\'2 | "Am" A,B,AA2 | "Dm" FAFE2 | "G7" D/B,/DG3 |\n"C" G,ECC\'2 | "Am" A,B,AA2 | "Dm" FAFE2 | "G7" G,B,/DA,3 :|\n|: "C" e2g2e2 | "G" d2G2B2 | "Am" A2c2A2 | "Em" B2E2G2 |\n"F" F2A2c2 | "C" E2G2c2 | "Dm" D2F2A2 | "G7" G/B,/DG3 :|' additional_kwargs={} example=False


### Parse the LLM output string into a Python dictionary

Let's create an LLM output JSON and use LangChain to parse that output.
Let's extract the melody data and format that output into a JSON format. 

In [57]:
description_schema = ResponseSchema(name="description",
                             description="Extract any sentences describing the melody \
                            an output them as a text.")
melody_pitch_duration_data_schema = ResponseSchema(name="melody_pitch_duration_data",
                                    description="""
    Extract the composed melody, and output them as a ABC music format.
    Replace the new line symbols by spaces and create a new line. 
    """)

response_schemas = [description_schema,
                    melody_pitch_duration_data_schema]

In [58]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [59]:
format_instructions = output_parser.get_format_instructions()

In [60]:
review_template = """\
For the following text, extract the following information:

description: Extract any sentences describing the melody \
an output them as a text.

melody_pitch_duration_data: Extract the composed melody, \
and output them as a ABC music format like this:

Extract the composed melody, and output them as a ABC music format.
Replace the new line symbols by spaces and create a new line. 

Format the output as JSON with the following keys:
description
melody_pitch_duration_data

text: {text}

{format_instructions}
"""

In [61]:
prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(text=output, 
                                format_instructions=format_instructions)

In [62]:
chat = ChatOpenAI(temperature=0.0, model=llm_model)

In [63]:
response = chat(messages)

In [64]:
print(response.content)

```json
{
	"description": "Sure, here is one more minuet:",
	"melody_pitch_duration_data": "X: 7 T: Bach Minuet 4 C: J. S. Bach (1685-1750) M: 3/4 L: 1/8 K: C |: \"C\" G,ECC'2 | \"Am\" A,B,AA2 | \"Dm\" FAFE2 | \"G7\" D/B,/DG3 | \"C\" G,ECC'2 | \"Am\" A,B,AA2 | \"Dm\" FAFE2 | \"G7\" G,B,/DA,3 :| |: \"C\" e2g2e2 | \"G\" d2G2B2 | \"Am\" A2c2A2 | \"Em\" B2E2G2 | \"F\" F2A2c2 | \"C\" E2G2c2 | \"Dm\" D2F2A2 | \"G7\" G/B,/DG3 :|"
}
```


In [65]:
output_dict = output_parser.parse(response.content)

In [66]:
output_dict

{'description': 'Sure, here is one more minuet:',
 'melody_pitch_duration_data': 'X: 7 T: Bach Minuet 4 C: J. S. Bach (1685-1750) M: 3/4 L: 1/8 K: C |: "C" G,ECC\'2 | "Am" A,B,AA2 | "Dm" FAFE2 | "G7" D/B,/DG3 | "C" G,ECC\'2 | "Am" A,B,AA2 | "Dm" FAFE2 | "G7" G,B,/DA,3 :| |: "C" e2g2e2 | "G" d2G2B2 | "Am" A2c2A2 | "Em" B2E2G2 | "F" F2A2c2 | "C" E2G2c2 | "Dm" D2F2A2 | "G7" G/B,/DG3 :|'}

In [42]:
melody_data = output_dict.get('melody_pitch_duration_data')
melody_data

'X: 7\nT: Bach Minuet 4\nC: J. S. Bach (1685-1750)\nM: 3/4\nL: 1/8\nK: C\n|: "C" G,ECC\'2 | "Am" A,B,AA2 | "Dm" FAFE2 | "G7" D/B,/DG3 |\n"C" G,ECC\'2 | "Am" A,B,AA2 | "Dm" FAFE2 | "G7" G,B,/DA,3 :|\n|: "C" e2g2e2 | "G" d2G2B2 | "Am" A2c2A2 | "Em" B2E2G2 |\n"F" F2A2c2 | "C" E2G2c2 | "Dm" D2F2A2 | "G7" G/B,/DG3 :|'

Save the string as a formatted ABC music notation file.

In [68]:
# Define the file name
file_name = "../data/processed/melody.abc"

# Write the melody data to an .abc file
with open(file_name, 'w') as file:
    file.write(melody_data)

In [ ]:
//////////////////////

In [ ]:
note_to_midi = config['note_to_midi']

In [ ]:
note_to_midi

In [ ]:
type(melody_data)

In [ ]:
from midiutil import MIDIFile

In [ ]:
def note_to_midi_mapping():
    """
    Create a dictionary to map note names to MIDI note numbers.
    
    Returns:
        dict: A dictionary mapping note names to MIDI note numbers.
    """
    # Define the mapping from note names to MIDI note numbers
    note_to_midi = config['note_to_midi']
    return note_to_midi

In [ ]:
def create_midi_file(notes, filename="melody.mid"):
    """
    Create a MIDI file from a list of notes.
    
    Parameters:
        notes (list): List of notes to include in the MIDI file.
        filename (str): Name of the output MIDI file.
    """
    # Initialize a MIDI file with one track
    midi = MIDIFile(1)
    
    # Set the instrument to Acoustic Grand Piano (program number 0)
    # The order is track, time, channel, program
    midi.addProgramChange(0, 0, 0, 0)
    
    # Add notes to the MIDI file
    for start_time, (pitch, duration) in enumerate(notes):
        midi.addNote(0, 0, pitch, start_time, duration, 100)
        
    # Save the MIDI file
    with open(filename, "wb") as output_file:
        midi.writeFile(output_file)


In [ ]:
melody_data